In [2]:
%matplotlib qt

from sklearnex import patch_sklearn
patch_sklearn()
import pandas as pd

df = pd.read_csv("../data/qbo_data.csv",index_col="run_id")
df = df.sort_values("qbo_period_std")
df = df[:65]

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [3]:
import numpy as np
X = df[["Bt","cw"]].to_numpy()
y_point_error = df[["qbo_period_std","qbo_amplitude_std"]].to_numpy()
y = df[["qbo_periods","qbo_amplitude_mean"]].to_numpy()

mean_y = np.mean(y,axis=0)
std_y = np.std(y,axis=0)
y_norm = (y - mean_y)/std_y
y_point_error = y_point_error/std_y

In [4]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

In [298]:
pca.explained_variance_

array([1.11274458, 0.91850542])

In [5]:
X_pca

array([[-2.06017912e+00,  1.63901913e-01],
       [ 8.50696623e-02,  6.23104274e-01],
       [-6.96296657e-02,  9.42022884e-01],
       [ 1.98075883e+00,  4.42730015e-01],
       [ 6.04820156e-01,  1.47438956e-01],
       [-4.02927468e-02,  1.39853805e+00],
       [ 5.42206903e-01,  4.12291106e-01],
       [ 6.95202955e-01,  9.67800997e-01],
       [-1.01686686e+00,  2.44795634e-01],
       [ 8.86969592e-01, -4.52532651e-01],
       [ 2.13888892e+00,  1.05687030e-01],
       [-2.53868171e-01, -6.59825489e-01],
       [-3.63271061e-01, -1.11998255e+00],
       [ 1.31136390e+00,  5.23158812e-01],
       [ 4.01806405e-02, -8.97235559e-01],
       [ 2.99215792e-01, -3.60654643e-01],
       [ 6.63187788e-01, -8.44762763e-01],
       [-4.32568310e-01, -1.57813680e+00],
       [ 9.08004762e-01,  8.24475069e-01],
       [-3.38742628e-01,  6.57204420e-02],
       [ 8.51972710e-01,  1.30904488e+00],
       [ 6.47177317e-01, -9.81865693e-01],
       [-1.93126901e+00,  6.47271817e-01],
       [ 8.

In [53]:
from sklearn.gaussian_process import GaussianProcessRegressor

gpr_period = GaussianProcessRegressor(alpha=(y_point_error[:,0])**2)
gpr_amplitude = GaussianProcessRegressor(alpha=(y_point_error[:,1])**2)
gpr_period.fit(X_pca,y_norm[:,0])
gpr_amplitude.fit(X_pca,y_norm[:,1])


GaussianProcessRegressor(alpha=array([0.        , 0.05614359, 0.05071146, 0.03369427, 0.05793871,
       0.15816117, 0.07520947, 0.03703018, 0.10870487, 0.06901974,
       0.03257372, 0.0523034 , 0.04739906, 0.04732584, 0.06270552,
       0.06617491, 0.04166084, 0.05919156, 0.1746527 , 0.08682533,
       0.18972332, 0.02037526, 0.11096865, 0.06729746, 0.0286461 ,
       0.04683844, 0.06160858, 0.05715145, 0.03597941, 0.08...
       0.02839342, 0.06052202, 0.05353102, 0.31069536, 0.03187466,
       0.04551048, 0.12801546, 0.04980584, 0.04364479, 0.08336826,
       0.09142026, 0.01448409, 0.06865047, 0.04861572, 0.06369928,
       0.03364536, 0.0986437 , 0.05727018, 0.07161853, 0.03180672,
       0.17901242, 0.11933718, 0.18275724, 0.07131104, 0.01872045,
       0.09918714, 0.28454018, 0.07868217, 0.07902559, 0.10776385,
       1.1082482 , 0.71114773, 0.12468573, 0.10788274, 0.07694426]))

In [54]:
import numpy as np
cw = np.linspace(5,70,1000)
Bt = np.ones(len(cw))*0.0043
x_samples = np.array([Bt,cw]).T
x_samples = scaler.transform(x_samples)
x_samples = pca.transform(x_samples)


In [55]:
samples,samples_std = gpr_period.predict(x_samples,return_std=True)
samples = samples*std_y[0] + mean_y[0]

In [56]:
std_y

array([3.5906523 , 8.42465129])

In [65]:
OBS_MEAN = 27.475168565819082
OBS_STD = 3.9921132486687365

OBS_AMP_MEAN = 26.68333333333334
OBS_AMP_STD = 2.7605353748060457

In [59]:
# import matplotlib.pyplot as plt
# fig,(ax1,ax2)= plt.subplots(nrows=1,ncols=2,figsize=(10,6))

# ax1.plot(cw,samples)
# ax1.fill_between(
#     cw,
#     samples - 1.96*samples_std*std_y[0],
#     samples + 1.96*samples_std*std_y[0],
#     color="tab:blue",
#     alpha=0.5,
#     label=r"95% confidence interval",
# )
# ax1.axhline(OBS_MEAN,color='black',label="True Observation")
# ax1.fill_between(cw,OBS_MEAN-OBS_STD,OBS_MEAN+OBS_STD,color='black',alpha=0.3)
# ax1.set_xlabel("Cw tropics")
# ax1.set_ylabel("QBO Period (months)")
# ax1.errorbar(X[:,1],y[:,0],yerr=y_point_error[:,0]*std_y[0],fmt='o',label="MiMA runs")
# ax1.set_ylim(10,40)
# ax1.legend()

# ax2.plot(cw,implausability)
# ax2.axhline(np.percentile(implausability,20),c='g',label=f"20th percentile implausability cut off ")
# ax2.legend()
# ax2.set_ylabel("1D Implausability")
# ax2.set_xlabel("cw")

Text(0.5, 0, 'cw')

### 3D Plots?

In [62]:
cw = np.linspace(5,70,100)
Bt = np.linspace(0.001,0.007,100)
X_Bt,Y_cw = np.meshgrid(Bt,cw)
x_samples = np.vstack((X_Bt.ravel(),Y_cw.ravel())).T
x_samples = scaler.transform(x_samples)
x_samples = pca.transform(x_samples)

In [63]:
x_samples

array([[-2.58971787e-03, -3.07633642e+00],
       [ 2.54074682e-02, -3.04833924e+00],
       [ 5.34046542e-02, -3.02034205e+00],
       ...,
       [-2.49802406e-01,  2.60233036e+00],
       [-2.21805220e-01,  2.63032754e+00],
       [-1.93808034e-01,  2.65832473e+00]])

In [82]:
period,period_std = gpr_period.predict(x_samples,return_std=True)
period = period*std_y[0] + mean_y[0]
period = period.reshape((100,100))
period_std = (period_std*std_y[0]).reshape((100,100))


amplitude,amplitude_std = gpr_amplitude.predict(x_samples,return_std=True)
amplitude = amplitude*std_y[1] + mean_y[1]
amplitude = amplitude.reshape((100,100))
amplitude_std = (amplitude_std*std_y[1]).reshape((100,100))

In [97]:
implausability_period = np.abs(period - OBS_MEAN)/np.sqrt(OBS_STD**2 + period_std**2)
implausability_amplitude = np.abs(amplitude - OBS_AMP_MEAN)/np.sqrt(OBS_AMP_STD**2 + amplitude_std**2)
implausability = np.sqrt(implausability_amplitude**2 + implausability_period**2)

In [98]:
imp_cutoff = implausability < np.percentile(implausability,20)

In [99]:
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(14,6))
CS = ax1.contour(X_Bt,Y_cw,amplitude,levels=25)
ax1.scatter(X[:,0],X[:,1],)
ax1.clabel(CS, CS.levels, inline=True,  fontsize=10)
ax1.set_ylabel("Cw")
ax1.set_xlabel("Bt")

CS2 = ax2.contour(X_Bt,Y_cw,period,levels=25)
ax2.scatter(X[:,0],X[:,1],)
ax2.clabel(CS2, CS2.levels, inline=True,  fontsize=10)
ax2.set_ylabel("Cw")
ax2.set_xlabel("Bt")

ax3.contour(X_Bt,Y_cw,implausability,levels=25)
ax3.contourf(X_Bt,Y_cw,imp_cutoff,alpha=0.3,cmap='RdYlGn')
ax3.scatter(X[:,0],X[:,1])
ax3.set_ylabel("Cw")
ax3.set_xlabel("Bt")

Text(0.5, 0, 'Bt')

In [52]:

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
ax.plot_surface(X_Bt,Y_cw,samples,cmap='viridis')
ax.set_xlabel("Bt")
ax.set_ylabel("Cw")
ax.set_zlabel("QBO Period (months)")

Text(0.5, 0, 'QBO Period (months)')

In [22]:
ax = plt.figure(figsize=(10,10)).add_subplot(projection='3d')
ax.plot_surface(X_Bt,Y_cw,samples,cmap='viridis')
ax.set_xlabel("Bt")
ax.set_ylabel("Cw")
ax.set_zlabel("QBO Period (months)")

array([[24.51892271, 24.51844809, 24.51797128, ..., 22.61459751,
        22.61078872, 22.60702863],
       [24.51722301, 24.51674178, 24.51625834, ..., 22.60285384,
        22.59903133, 22.59525775],
       [24.5155125 , 24.51502464, 24.51453456, ..., 22.59108812,
        22.5872519 , 22.58346481],
       ...,
       [ 7.20922186,  7.1788672 ,  7.14886289, ..., 23.70143329,
        23.69731721, 23.69328177],
       [ 7.23238639,  7.20207043,  7.17210431, ..., 23.70608979,
        23.70194998, 23.69789081],
       [ 7.25589211,  7.22561564,  7.19568848, ..., 23.71072927,
        23.70656597, 23.70248331]])